In [1]:
import os
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index.prompts import PromptTemplate
from llama_index import download_loader
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import MetadataExtractor, TitleExtractor, SummaryExtractor, QuestionsAnsweredExtractor, KeywordExtractor
from llama_index.llms import OpenAI
from llama_index import StorageContext, load_index_from_storage
from glob import glob
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.text_splitter import TokenTextSplitter, SentenceSplitter
import torch

# Data

In [2]:
docs = SimpleDirectoryReader(input_dir='data_txt').load_data()
len(docs)

4

In [3]:
# llm_gpt = OpenAI(temperature=0, model='gpt-3.5-turbo')

# NODE_TEMPLATE_RUS = "Контекст: {context_str}. Дай заголовок, который суммирует все уникальные объекты, названия или темы, встречающиеся в контексте. Заголовок: "

# COMBINE_TEMPLATE_RUS = "{context_str}. Основываясь на приведенных выше заловоках кандидатов и содержании, каким общим заголовком их можно назвать? Заголовок: "

# DEFAULT_NODE_TEXT_TEMPLATE_RUS = "[Выдержка из документа]\n{metadata_str}\nВыдержка:\n-----\n{content}\n-----\n"

# DEFAULT_SUMMARY_EXTRACT_TEMPLATE = """\
# Вот содержание этого раздела:
# {context_str}

# Кратко изложите ключевые темы и сущности раздела. \

# Краткое изложение: """

# DEFAULT_QUESTION_GEN_TMPL = """\
# Вот контекст:
# {context_str}

# Учитывая контекстуальную информацию, сгенерируйте {num_questions} вопросов, на которые в этом контексте могут быть даны конкретные ответы, которые вряд ли можно найти где-либо еще.

# Также могут быть предоставлены краткие сведения об окружающем контексте более высокого уровня. Попробуйте использовать эти сведения, чтобы лучше сформулировать вопросы, на которые можно ответить в данном контексте.

# """

# metadata_extractor = MetadataExtractor(
#   extractors=[
#   # TitleExtractor(nodes=10, llm=llm_gpt, node_template=NODE_TEMPLATE_RUS, combine_template=COMBINE_TEMPLATE_RUS),
#   KeywordExtractor(keywords=5, llm=llm_gpt),
#   # SummaryExtractor(prompt_template=DEFAULT_SUMMARY_EXTRACT_TEMPLATE, summaries=["self"]),
#   # QuestionsAnsweredExtractor(questions=5, prompt_template=DEFAULT_QUESTION_GEN_TMPL)
#   ],
#   node_text_template=DEFAULT_NODE_TEXT_TEMPLATE_RUS
# )

In [4]:
tok = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru", use_fast=False)
text_splitter = TokenTextSplitter(
    separator="\n\n", chunk_size=512, chunk_overlap=128)

parser = SimpleNodeParser.from_defaults(
    # metadata_extractor=metadata_extractor,
    text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(docs)
len(nodes)

411

In [5]:
import re

for node in nodes:
    node.text = re.sub("\\n", " ", node.text)
    node.text = re.sub(" +", " ", node.text)

In [6]:
nodes[1].text

'3. Принцип практики Все гипотезы мы проверяем на практике. 4. Принцип непрерывных улучшений Улучшаем себя на основе регулярной обратной связи и рефлексии. 5. Принцип равноценности Каждый на равных может участвовать в подготовке решений, которые его затрагивают. 6. Принцип «все видят все» Вся информация в ГК общедоступна, если нет причин для конфиденциальности.'

In [7]:
# os.environ['OPENAI_API_KEY'] = 'None'

# Model

In [8]:
promt_llama_1 = "Ответь на вопрос: {query_str}"

sp = "Ты умный ассистент которого зовут Хьюстон. Ты любишь отвечать на вопросы пользователей."

MODEL_NAME = "IlyaGusev/saiga2_7b_lora"

config = PeftConfig.from_pretrained(MODEL_NAME)
llm = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto"
)
llm = PeftModel.from_pretrained(llm, MODEL_NAME, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

llm = HuggingFaceLLM(
    model=llm,
    tokenizer=tokenizer,
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=promt_llama_1,
    system_prompt=sp,
    device_map="auto",
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)
service_context = ServiceContext.from_defaults(
    chunk_size=256, llm=llm, embed_model="local:cointegrated/LaBSE-en-ru")
# service_context = ServiceContext.from_defaults(chunk_size=512, llm=llm, embed_model=llm)

torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
index = VectorStoreIndex(nodes=nodes, service_context=service_context)

In [10]:
# index.storage_context.persist()

In [2]:
# storage_context = StorageContext.from_defaults(persist_dir="./storage")
# index = load_index_from_storage(storage_context)

******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******
******
Could not load OpenAIEmbedding. Using HuggingFaceBgeEmbeddings with model_name=BAAI/bge-small-en. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  5120

In [11]:
# response_mode=compact, tree_summarize
query_engine = index.as_query_engine(similarity_top_k=5)

In [22]:
res = query_engine.query(
    "Какой размер пособия по временной нетрудоспособности?")
print(res.response)

/home/pc/Desktop/llamaindex/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


60% среднего заработка, если страховой стаж от 6 месяцев до 5 лет; 80% от 5 до 8 лет; 100% более 8 лет.


In [13]:
res.source_nodes

[NodeWithScore(node=TextNode(id_='ddd64bb6-e737-4892-8dfa-a793f175d8f1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0d58345f-085f-4e24-b536-32817d923955', node_type=None, metadata={}, hash='b8b111adb2a048773fd29b63c0f90a5ab2fb4c72afbfc877931c5985b3f58611'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a8fedc2a-a244-42d5-932d-2fcddaf44e19', node_type=None, metadata={}, hash='184aded1e7f45f2ecdfce07ed9e93a50d554d3cbfd2f1970250826c09e7575d2')}, hash='96eb63e3751c4df96d653b36bebe6e53cc76b1e4c6b4aa55ce449ef765801ee0', text='Больничные Размер пособия по временной нетрудоспособности (больничного) зависит от страхового стажа сотрудника: • если страховой стаж от 6 месяцев до 5 лет, то ему оплачивается 60% среднего заработка; • от 5 до 8 лет — 80%; • более 8 лет — 100%. • если стаж сотрудника менее 6 месяцев, то пособие по временной нетрудоспособности выплачивается 

In [14]:
res = query_engine.query("Что такое Смартократия?")
res.response.strip()

'Смартократия — это форма управления, которая основывается на самоорганизации и ролях. Она стремится к максимальному использованию возможностей каждого сотрудника и созданию культуры, где каждый может вносить свой вклад в развитие компании. Смартократия также призывает к полной открытости информации и сотрудничеству между всеми сотрудниками.'

In [15]:
res = query_engine.query("На каких принципах базируется Смартократия?")
res.response

'1. Принцип результативности 2. Принцип консента 3. Принцип практики 4. Принцип благодарности за обратную связь 5. Цифровая культура 6. Информационное поле'

In [20]:
res = query_engine.query("Как оформить больничный?")
res.response.strip()

/home/pc/Desktop/llamaindex/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'1. Сначала необходимо обратиться в медицинский центр и получить медицинское заключение.\n2. Затем, следует обратиться в свое работодательское учреждение и подать заявление на больничный.\n3. В заявлении необходимо указать причину больничного и срок его продления.\n4. После получения заявления, работодательское учреждение выдаст больничный на определенный срок.\n5. В течение больничного, работник должен следовать инструкциям своего работодателя и не нарушать условия больничного.\n6. После окончания больничного, работник должен обратиться в свое работодательское учреждение и подать заявление на возвращение на работу.'

In [ ]:
query_engine.

In [ ]:
res.source_nodes

[NodeWithScore(node=TextNode(id_='2f3df848-8895-402f-bf6d-a7e01c43f0d9', embedding=None, metadata={'document_title': 'Принципы и организация смартократии в ГК Smart Consulting', 'excerpt_keywords': 'больничные, пособие, нетрудоспособность, стаж, доступы'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='55cecc7a-ca6e-4ffd-a1e2-7a574a3e9c76', node_type=None, metadata={}, hash='3ed2f85c90333d4e07ef95a67bf059a7ee13f1928955fa54fd858b53132b5bf8'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3fb6c882-174a-4466-b030-19b06a61adae', node_type=None, metadata={}, hash='cb50dd386db474646e8c07644e45be7d1abfb0e29f93adeeceac4cf7f9360023')}, hash='ff37d9fe312839dd65fbf7ec5ea73203ddb2eb60f6cf1c863b782bbabd5a45fb', text='Больничные\nРазмер пособия по временной нетрудоспособности (больничного) зависит от страхового стажа сотрудника:\n- если страховой стаж от 6 месяцев до 5 лет, то ему оплачивается 60% средн